<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup-Data-Pre-Processing" data-toc-modified-id="Setup-Data-Pre-Processing-0"><span class="toc-item-num">0&nbsp;&nbsp;</span>Setup Data Pre-Processing</a></span><ul class="toc-item"><li><span><a href="#Raw-Data-to-Feature-Data" data-toc-modified-id="Raw-Data-to-Feature-Data-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Raw Data to Feature Data</a></span></li><li><span><a href="#Tensor-Data-Structure-Input" data-toc-modified-id="Tensor-Data-Structure-Input-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Tensor Data-Structure Input</a></span></li></ul></li><li><span><a href="#Architecture-Basics" data-toc-modified-id="Architecture-Basics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Architecture Basics</a></span><ul class="toc-item"><li><span><a href="#Gated-CNN" data-toc-modified-id="Gated-CNN-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Gated CNN</a></span><ul class="toc-item"><li><span><a href="#GCNN-1D-Time-Series" data-toc-modified-id="GCNN-1D-Time-Series-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>GCNN 1D Time Series</a></span></li><li><span><a href="#GCNN-1D-Residuals" data-toc-modified-id="GCNN-1D-Residuals-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>GCNN 1D Residuals</a></span></li><li><span><a href="#GCNN-2D" data-toc-modified-id="GCNN-2D-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>GCNN 2D</a></span></li></ul></li></ul></li><li><span><a href="#Hyper-Param-Protocol-(hpp)" data-toc-modified-id="Hyper-Param-Protocol-(hpp)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Hyper Param Protocol (hpp)</a></span></li></ul></div>

In [95]:
# Imports e definicoes auxiliares, TODO - fazer module apenas para estas definicoes auxiliares
from IPython.display import Math
#!pip install librosa
import tensorflow as tf
from tensorflow.python import debug as tf_debug
from tensorflow.python import util
from tensorflow.python.framework import ops as tf_ops
tf.summary.initialize = tf.contrib.summary.initialize
tf.variable_scope = tf.compat.v1.variable_scope

#  Importing helper functions for spectrogram, comp graph viz and helper functions to calculate filters and device scopes
from spect import *
from cgraphviz import *
from tfhelper import *

import numpy as np
from six import string_types
import argparse
import functools
from numpy import asarray, array, ravel, repeat, prod, mean, where, ones
import sys
import copy
import re
import os

import matplotlib.pyplot as plt
from tensorflow.python.client import timeline
    
# In TensorFlow a device name is a string of the following form:
#   /job:<name>/replica:<replica>/task:<task>/device:<type>:<device_num>
#
# <name> is a short identifier conforming to the regexp
#     [a-zA-Z][_a-zA-Z]*
# <type> is a supported device type (e.g. 'cpu' or 'gpu')
# <replica>, <task>, <device_num> are small non-negative integers and are
# densely allocated (except in tests).
#
# For some purposes, we also allow device patterns, which can specify
# some or none of the specific fields above, with missing components,
# or "<component>:*" indicating "any value allowed for that component.
#
# For example:
#   "/job:param_server"   - Consider any devices in the "param_server" job
#   "/device:cpu:*"       - Consider any cpu devices in any job/task/replica
#   "/job:*/replica:*/task:*/device:cpu:*"  - Consider any cpu devices in any
#                                             job/task/replica
#   "/job:w/replica:0/task:0/device:gpu:*"  - Consider any gpu devices in
#                                             replica 0, task 0, of job "w"

###
## Danijar option to define scope with decorators
#

def doublewrap(function):
    """
    A decorator decorator, allowing to use the decorator to be used without
    parentheses if no arguments are provided. All arguments must be optional.
    """
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator


@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    """
    A decorator for functions that define TensorFlow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result so that operations are added to the graph only once.
    The operations added by the function live within a tf.variable_scope(). If
    this decorator is used with arguments, they will be forwarded to the
    variable scope. The scope name defaults to the name of the wrapped
    function.
    """
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__
    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator
#####
####
##
#

class AttrDict(dict):
    
  __getattr__ = dict.__getitem__
  __setattr__ = dict.__setitem__      

# Setup Data Pre-Processing 

## Raw Data to Feature Data

In [ ]:
# Raw data to Log-Spectogram 
# Read data from the hard-drive
spect_list = []
raw_list = []
nomes = [f for f in os.listdir('/home/penalvad/stattus4/benchfull/')]
label_list = []
c = 0
b = 0 
m = 0
for i in range(len(nomes)):
    #ignore .mat
  if nomes[i].find('.mat') == -1 and (nomes[i].find('sv') != -1 or nomes[i].find('SV') != -1) and c < 10:    
    
    data,fs = librosa.load('/home/penalvad/stattus4/benchfull/'+nomes[i],sr=None)
    spec = subbed_spect(data,fs,plot=False)
    m = np.shape(spec[0])
    
    if m[0] > 600 and m[1] > 50:      
      spect_list.append( spec[0][:600,:50] )
      label_list.append(nomes[i][0:2])
      c += 1        
    #raw_list.append((data,fs))
    

  if nomes[i].find('.mat') == -1 and (nomes[i].find('cv') != -1 or nomes[i].find('CV') != -1 )and b < 10:
    
    data,fs = librosa.load('/home/penalvad/stattus4/benchfull/'+nomes[i],sr=None)
    spec = subbed_spect(data,fs,plot=False)
    m = np.shape(spec[0])
    
    if m[0] > 600 and m[1] > 50:
      spect_list.append( spec[0][:600,:50] )
      label_list.append(nomes[i][0:2])
      b += 1             
    #raw_list.append((data,fs))    

## Tensor Data-Structure Input

In [ ]:
# Data to Tensor Data for consumption on Tensorflow models
# TODO - equalize data size for downsampled raw data
# - On supposition that the data size is the same for all time series, if not is necessary to assert it.
# - The same is true for spect, however downsampled spect may lead to smaller ts.
#ts  = sample_gaussmix(shape=(10000,), mean_base=0.5, var_base=0.15, random_mods=True, num_mods=50)

batch = len(spect_list)
#data_size_ts = len(raw_list[0][0])
data_size_image = (len(spect_list[0][:,0]),len(spect_list[0][0,:]))
channels = 1
#print(data_size_ts)

#data_tensor_1D = np.ndarray( shape=(batch,data_size_ts,channels) )
data_tensor = np.ndarray( shape=(batch,data_size_image[0],data_size_image[1],channels) )

for i in range(batch):

  #data_tensor_1D[i,:,0] = raw_list[i][0]
  data_tensor[i,:,:,0] = spect_list[i][:,:]
    
np.shape(data_tensor)

# Architecture Basics

In [195]:
# Definitions: Build computation graph and run session

#####
###  Inicio de refatoracao usando principios de design patterns. 
# TODO: pesquisar mais e mais e mais a fundo o design do Tensorflow pra seguir de perto.

from enum import Enum

class BlockTypes(Enum):
  GCNN2D = 'gcnn2d'
  SOFTMAX = 'softmax'
  CROSSENTROPY = 'loss-crossentropy'
  INPUT = 'signal_in'
  REDUCEMEAN = 'reducemean'
  
class ArchBlocks:

  def __add__(self, block):
    pass
    
class Architecture(ArchBlocks):
  '''
    aa
  '''
  def __init__(self, **kwargs):
    
    self.modules = {'0':[ArchBlocks.INPUT,0,0,[],[]]}

  def __add__(self, archblock1):
    pass
    #self.modules = archblock1
    
    
  def _block_specification(self,key):    
    pass

class Stattus4NeuralNetAPI:
    
  def __init__(self, datatype, datasocket = False, **kwargs):
    '''
     Input
     
     datatype: Audio, Pressure, Flow.
     datasocket: if True tries to fetch data from a database and push output to database 
     pre-configured through kwargs, if config is not provided then tries to read from datafeeder or dump to local file-system.   
     
     Optional
     num_of_channels: number of channels for Audio data, if not given get only the first audio and discard the others.    
    '''
    self.datatype = datatype
    self.datasocket = datasocket
    
  def setup_hyperparameters(self, architectures):
    '''
      Input
      
      architectures: Architecture object specifying archetypical structure to be investigated for its hyperparams.

      Return a dictionary of the given 
    '''
    pass

#######
#######
 ####
  ### 
   #  

#  FUTURE: Name/Var Scope handler. This class/template must be used to implement logic of names 
# according to the architecture block connection. E.G. if a block is just the same from the other in the
# sequence them it must be named as if is a deep part of the same architecture blocks. If it is a ramification
# of the given below but the same arch them he must be named with another tag.
# When duplicating some block to architectonic use, E.G. if you need to frame your data or feature, them each
# frame must have an coeherent name space block, like the today ordimatlicly done for repeated name space block,
# but this must be automaticly done for identical built blocks of the model, without the need to explicity code it.
# Names must be given as unique ID according to te input-output architecture connection, if it has the same IO from previous Block in the sequence,
# then each Input-ARchBlock-Output that is IDentical must have the same name with the appropriated tag for deepnes in the sequence of blocks
# Automatic update of the name scopes according to the architecture.
# Dont change Identical sequential IO Name spaces  

####
### General purpose builder function, to be substituted in FUTURE refatoration mentioned above
## Serve as Computation Graph builder and saver (through session saver), editor and Computational Graph runner.
#
class Builder:  
  
  def __init__(self, **kwargs):
    '''
      Build a Tensorflow computational graph, from scratch.      
        
      Requires signal input parameters such:
      - batch: for batch size
      - datasize: give a tuple with the signal input shape
      - channels: self explain.
      
      
      Note 0: You can always call Builder.get_directives(archblocktype) to know the inner params to the building block architecture.
      Note 1: The object guard reference for each op in a dictionary namescopes with the name of each block.
      TODO - export serialized file containing the model (e.g. .pb)
    '''  
    
    #Use attribute dict ##FUTURE: use arch block class with enums specifing the blocks
    
    #Provisional name tracking for this network.
    
    self.namescopes = AttrDict()    
    self.architectures = AttrDict()
    self.architectures.gcnn1d = self._gcnn1d
    self.architectures.gcnn2d = self._gcnn2d
    self.architectures.reducemean = self._reducemean
    self.architectures.losscrossentropy = self._losscrossentropy
    self.architectures.softmax = self._softmax
    self.architectures.residualgcnn1d = self._residualgcnn1d
    
    
    #TODO also use attribute dict, each arch has it own directive dict in with each directive has its shape has value
    ##FUTURE: use arch block class with enums specifing the blocks
    ##   Auto compile directives from json and query parameters,
    ## return requisition or error if parameter not given or is not from appropriated shape and type. 
    self.directives = {
        'gcnn2d': {'channels_out':1,
                   'filter_size':2},
        'reducemean':{'None':None},
        'losscrossentropy':{'num_labels':1,
                            'batch':1,
                            'learningrate':1},
        'softmax':{'num_labels':1},  
        'residualgcnn1d':{'channels_out':1,
                          'filter_size':1},
        'gcnn1d':{'channels_out':1,
                   'filter_size':2
                 },
        'signalimg':{'batch':1,
                   'datasize':2,
                   'channels':1,
                   'dtype':1
                    }    
    }
    
    tf.reset_default_graph()
    # TODO-Define Specific Exception Handler for the Input
    try:
        
      self.batch = kwargs['batch']
      self.datasize = kwargs['datasize']
      self.channels = '' or kwargs['channels']
      self.dtype = kwargs['dtype']
    
    except KeyError:
        
      sys.exit('Signal Input Not Defined Error')      
    
    try:
        
      self.num_input = kwargs['num_input']
    
    except:
        
      self.num_input = 1
      
    self.graph = tf.Graph()
    self.signal_in = []  
    # FUTURE: this will go to another class
    self.arch_blocks = {}    
    
  def __call__(self, **kwargs):        
   
    try:
      isinput = kwargs['isinput']
      if type(isinput) != type(bool()):
        isinput = True
    except:
      isinput = False
    
    try:
      lastnamescope= kwargs['lastnamescope']
      if type(lastnamescope) != type(bool()):
        lastnamescope= False
    except:
      lastnamescope= True       


    if isinput:
      with self.graph.as_default():
      # TODO - Different input shapes and data types (char,string,hexa,float,int...)                      
        if self.num_input > 0:
          self.signal_in.append(tf.placeholder(self.dtype,shape=(self.batch,self.datasize[0],self.datasize[1],self.channels),name='signal_in'))           
          self.namescopes['signal_in'] = self.signal_in
          self.num_input -= 1
        
      return self.signal_in[-1]
    elif lastnamescope:
      with self.graph.as_default():  
        return self.graph.get_tensor_by_name(self.graph.get_operations()[-1].name+':0')
    else:
      try:
        namescope = kwargs['namescope']
        with self.graph.as_default():
          with tf.name_scope(namescope+'/'):
            opa = 0
            for op in self.graph.get_operations():
              if op.name.split('/')[0] == namescope:
                opa = op 
            return self.graph.get_tensor_by_name(opa.name+':0')
      except:        
        sys.exit("Namescope not given to the input of architectures")            
        
  def _compute_membatchsize(self):
    c = 0
    for key in self.namescopes.keys():
      for tensor in self.namescopes[key]:          
        if type(tensor).__name__ == 'Tensor':
          
          c = c + tensor.get_shape().num_elements()          

    return c*8.0/(1024.0*1024.0*1024.0)  

  def _compute_memusage(self):

    c = 0 
    for v in buildgraph.graph._collections['variables']:      
      c = c + v.shape.num_elements()
    return c*4.0/(1024.0*1024.0*1024.0)      
        
  def estimate_memusage(self,limit = 12.0):
    self.mem_usage = self._compute_membatchsize() + self._compute_memusage()
    if self.mem_usage > limit:
      raise MemoryError('The model will allocate more memory, {}, than the given limit.'.format(self.mem_usage))    
    
  def assertive(self, archname):
    if hasattr(self, 'deepness') != True or getattr(self, 'deepness') == None:
      raise NameError('{} deepness not setted'.format(archname))
    
    if hasattr(self, 'numblock') != True or getattr(self, 'numblock') == None:
      raise NameError('{} block number not setted'.format(archname))    

  def _gcnn2d(self, **kwargs):
    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']
          
    except KeyError:
        
      sys.exit('Parameters Not Defined Error')  
    
    signal_in = self(**kwargs)
     
    ####            
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias                    
    self.assertive('gcnn2d')
    
    with self.graph.as_default():
      with tf.variable_scope('gcnn2d'+self.numblock+self.deepness[0]):
        
        self.namescopes['gcnn2d'+self.numblock+self.deepness[0]] = []    
        
        with self.graph.device(dev_selector(arg1='foo')('gcnn2d')):
          conv_linear = tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv_linear', use_bias=True, kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in)
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(conv_linear)            

        with self.graph.device(dev_selector(arg1='foo')('gcnn2d')):
          conv_gate = tf.sigmoid(tf.keras.layers.Conv2D( channels_out, filter_size, padding='valid', name='conv', use_bias=True,  kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in),name='conv_sigmoid')          
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(conv_gate)  
            
        with self.graph.device(dev_selector(arg1='foo')('gcnn2d')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions') 
          self.namescopes['gcnn2d'+self.numblock+self.deepness[0]].append(gated_convolutions)
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
    
  def _gcnn1d(self, **kwargs):

    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']

    except Exception:
        
      sys.exit('Parameters Not Defined Error')
     
      signal_in = self(**kwargs)
        
    # postfix = self.get_namepostfix('gccn1d',**kwargs)
      with self.graph.as_default():
        with tf.variable_scope('gccn1d'):#+postfix):
                             
          with self.graph.device(_dev_selector(arg1='foo')):
            conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(signal_in)
        
          with self.graph.device(_dev_selector(arg1='foo')):
            conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(signal_in),name='conv_sigmoid')
        
          with self.graph.device(_dev_selector(arg1='foo')):
            gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions') 

  def _residualgcnn1d(self, **kwargs):  
    
    try:
        
      channels_out = kwargs['channels_out']
      filter_size = kwargs['filter_size']
        
    except KeyError:
        
      sys.exit('Parameters Not Defined Error')  
  
    signal_in = self(**kwargs)
                               
    
      ####
      ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias
                             
        # postfix = self.get_namepostfix('gccn1d',**kwargs)
    with self.graph.as_default():
      with tf.variable_scope('residualgccn1d'):#+postfix):                            
                    
        with self.graph.device(_dev_selector(arg1='foo')):            
          conv_linear = tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv_linear', use_bias=True)(signal_in)
        with self.graph.device(_dev_selector(arg1='foo')):        
          conv_gate = tf.sigmoid(tf.keras.layers.Conv1D( channels_out, filter_size, padding='causal', name='conv', use_bias=True )(signal_in),name='conv_sigmoid')
        with self.graph.device(_dev_selector(arg1='foo')):
          gated_convolutions = tf.multiply(conv_linear,conv_gate,name='gated_convolutions')
        
      # Input channels must be the same size of the convolution channels output (ie number of filters applied)
      with self.graph.device(_dev_selector(arg1='foo')):
        residual = tf.add(gated_convolutions,signal_in,name='residual')        

  def _softmax(self, **kwargs):        

    try:        
      num_labels = kwargs['num_labels']      
    except Exception:        
      sys.exit('Parameters Not Defined Error')       
    
    signal_in = self(**kwargs)    
    self.assertive('softmax')   
        
    with self.graph.as_default():
      with tf.variable_scope('softmax'+self.numblock+self.deepness[0]):
        
        self.namescopes['softmax'+self.numblock+self.deepness[0]] = []          
        
        with self.graph.device(dev_selector(arg1='foo')('softmax')):
          logits = tf.contrib.layers.fully_connected(signal_in, num_labels, activation_fn=None, normalizer_fn=None, normalizer_params=None, weights_initializer=tf.initializers.lecun_normal(seed=731), weights_regularizer=None, biases_initializer=tf.initializers.lecun_normal(seed=777), biases_regularizer=None, reuse=None, variables_collections=None, outputs_collections=None, trainable=True, scope='logit')
          self.namescopes['softmax'+self.numblock+self.deepness[0]].append(logits)
            
        with self.graph.device(dev_selector(arg1='foo')('softmax')):
          softmax = tf.nn.softmax(logits,axis=0)            
          self.namescopes['softmax'+self.numblock+self.deepness[0]].append(softmax)                      
        
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None         

  def _losscrossentropy(self, **kwargs):    
    
    try:        
      num_labels = kwargs['num_labels']
      batch = kwargs['batch']   
      learningrate = kwargs['learningrate']
    except Exception:        
      sys.exit('Parameters Not Defined Error')
    # TODO SIGNAL IN de acordo com arquitetura.
    signal_in = self(**kwargs)            
    ####        
    ## Keras convolutions. Classes, so dont behave like functions but outputs Tensor, use its functions to query the variables filters and bias   
    self.assertive('losscrossentropy') 
    with self.graph.as_default():
      with tf.variable_scope('losscrossentropy'+self.numblock+self.deepness[0]):        

        self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]] = []        
        self.l_vec = tf.placeholder(tf.float32,(batch,num_labels), name='labels')
        with self.graph.device(dev_selector(arg1='foo')('losscrossentropy')):
          loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0)(self.l_vec, signal_in)         
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(loss)
          self.loss = loss

        with self.graph.device(dev_selector(arg1='foo')('losscrossentropy')):
          self.minimize_op = tf.train.AdamOptimizer(learning_rate=learningrate).minimize(self.loss)
          self.namescopes['losscrossentropy'+self.numblock+self.deepness[0]].append(self.minimize_op)
            
        self.init_op = tf.global_variables_initializer()            
        
        if 'limit' in kwargs:          
          self.estimate_memusage(limit = kwargs['limit'])
        else:
          self.estimate_memusage()
  
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
        
  def _reducemean(self, **kwargs):
     
    scope_tensor_name = find_softmax(self.graph)
    inputs = get_tensor_list_from_scopes(self.graph, scope_tensor_name)    
    
    self.assertive('reducemean')
    
    with self.graph.as_default():
      with tf.variable_scope('reducemean'+self.numblock+self.deepness[0]):

        self.namescopes['reducemean'+self.numblock+self.deepness[0]] = []
  
        with self.graph.device(dev_selector(arg1='foo')('reducemean')):
          rm = tf.math.reduce_mean( inputs, axis=1 )
          self.namescopes['reducemean'+self.numblock+self.deepness[0]].append(rm)
          rmshape = tf.shape(rm)
          rmean = tf.reshape(rm, shape=(rmshape[0], rmshape[2]))   
          self.namescopes['reducemean'+self.numblock+self.deepness[0]].append(rmean)
            
        self.deepness.pop(0)
        if len(self.deepness) == 0:
          self.deepness = None
  
  def get_directives(self,arch):
      
    # TODO: Use as automatic requirement builder from file, maybe json.
    # Provisory setup for namespace tags
    #self._nametags = {arch:{}}
    return self.directives[arch]     
      
  # provisory configuration for next sequence of build module names, FUTURE: Hyper-Param-Protocol will learn
# the names automatically from given configuration or from the graph architecture updating.
  def config_block_name(self,deepness=0,numblock=0):
    '''
      Helper function to configure block architecture names.
      Inputs are deepness of the block, if there is any hidden layer, and the block number if there is more than one block with this architecture in the graph.          
    '''
    if deepness < 0:
      deepness = 0
      self.deepness = ['']
    else:  
      self.deepness = ['']
      for i in range(deepness):
        self.deepness.append('_d'+str(i))
    if numblock <= 0:
      self.numblock = ''
    else:
      self.numblock = '_'+str(numblock)
        
  # Definitions: Build computation graph and run session
  def build_graph_module(self, arch, show_cgraph=True, verbose=False, **kwargs):
    '''
      Build a Tensorflow computational graph.building_function
      Currently supports 1-D gcnn or residual 1-D gcnn for unidimensional variable and 2D gcnn for bidimensional frame data.
    
      tensor_scope_name:  name scope of the architecture unit to build the next module
    
        
      kwargs:    
      filter_size: conv filter size. Scalar number for time series, bidimensional tuple for image.
      channels_out: Number of channels for output convolution.
    
      TODO - export serialized file containing the model (e.g. .pb)
    '''
    
    self.architectures[arch](**kwargs) 
    
    with self.graph.as_default(): 
      if show_cgraph:
        print_display_cgraph(self.graph, verbose=verbose)
        
  def _save_trainable_vars(self, blockname):
    with self.graph.as_default():   
        
        
      saver = tf.train.Saver(var_list=self.graph.get_collection('trainable_variables') )      
      tf.global_variables_initializer()
        
      sess = tf.Session(graph=self.graph)      
      sess.run(self.graph.get_operations()[-1])   
      saver.save(sess, os.getcwd()+'/'+blockname)
      #saver.export_meta_graph(filename=blockname+'.constructor', collection_list='trainable_variables', export_scope=None, strip_default_attrs=False)
      self.arch_blocks[blockname] = blockname
        
  def define_block(self,blockname):             
    
    self._save_trainable_vars(blockname)    
    
    graph = tf.Graph()
    with graph.as_default():                                      
      sess = tf.Session(graph=graph)  
      
      new_saver = tf.train.import_meta_graph(os.getcwd()+'/'+self.arch_blocks[blockname]+'.meta', import_scope = blockname)             
      new_saver.restore(sess, os.getcwd()+'/'+self.arch_blocks[blockname])           
      ####  
      ## Redefinir signal_in dos blocos utilizados para construir este bloco através do dict namescopo
      ## para que run_cgraph consiga achar inputs  
      # 
        
      ####
      ## Signalin handling provisorio, fazer um modulo apenas para lidar com signal
      #        
      self.graph = graph
      self.signal_in = []
      self.signal_in.append(self.graph.get_tensor_by_name(blockname+'/signal_in:0'))   

  # Build block from name in the top level of the blocks, this is a previous version that take names inside namescope dict and build new block  
  def from_block(self,archblockname,tag = ""):
        
    with self.graph.as_default():              
      sess = tf.Session(graph=self.graph)  

      new_saver = tf.train.import_meta_graph(os.getcwd()+'/'+self.arch_blocks[archblockname]+'.meta', import_scope = archblockname+tag)        
      new_saver.restore(sess, os.getcwd()+'/'+self.arch_blocks[archblockname])            
      
      self.signal_in.append(self.graph.get_tensor_by_name(archblockname+tag+'/signal_in:0'))
      self.num_input -= 1
    
  def run_cgraph(self, feed_dict, op_to_run = '', number_of_runs = 10, mode = 'feed forward', new_session=True, output_log = True, verbose=True):
    '''
     Run Session
     TODO - session log output
     TODO - in the mode feed forward, only get tensor value for input, in the mode minimize run the last op that represents minimization     
     TODO - make op to run optional depending on modes
     TODO - getvariable by namescope
    '''       
    if new_session:
      session = tf.Session(graph=self.graph)  
      #session = tf_debug.TensorBoardDebugWrapperSession(session,"grpc://localhost:6064")
    else:
      session = tf.get_default_session()
    
    with session.as_default():
    #with tf.device(_dev_selector(''))
      feed_dict = feed_dict
      options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
      run_metadata = tf.RunMetadata()
      opval = []
      lossval = np.zeros(number_of_runs)
    
    # Search for Initialization: TODO binary search, using brute force ... 
      session.run(self.init_op)
      print('Global Variables Initialized')
        
    # Perform  gradient descent steps or feed forward according to the Operation Required   
      
      self.op_to_run = op_to_run
      #with tf.summary.FileWriter('/home/penalvad/stattus4/stattus4-audio-models/notebooks/',graph=self.graph,session=session) as writer:
        #tf.summary.initialize(graph=self.graph,session=session)               
             
      for step in range(number_of_runs):
        
        if mode == 'minimize':
          loss_value,_ = session.run([self.loss,self.minimize_op], feed_dict=feed_dict)
          lossval[step] = loss_value
          if step % 1 == 0 and verbose:        
            print("Step:", step, " Loss:", lossval[step])           

          if step % 5 == 0 and step != 0:          
            loss_diff = np.diff(lossval[np.nonzero(lossval)])
            print("Mean Loss Growth ", np.mean(loss_diff) )        
            
        if self.op_to_run != '':            
          op_value = session.run(self.op_to_run, feed_dict=feed_dict)             
          opval.append( op_value )

      #gated_convs_value = session.run(graph.get_tensor_by_name('gated_convolutions:0'), feed_dict = feed_dict, options=options, run_metadata=run_metadata)
    return opval

##  Gated CNN
  Gated CNN is a doubled CNN in whom one of the convoluted signals does the role of opening/closing the network, giving an **Attention Mechanism** to the convolution, for being activated by a sigmoid.
  It gives non-vanishing gradient, since the multiplication rule for the derivative applies, also, applies gradient to the linear convoluted part.
  

### GCNN 2D
  This unit uses sigmoid function in all frame pixels and do element-wise multiplication. Use it for images or time-series TF Features frame. The standard convolution mode is Valid. The output size of the convolution is given by:
 
 Padding Valid
\begin{align}
dimemsion output size = \ceil{ \frac{dim size - (kernel dim size - 1)*dilation rate}{stride} }
\end{align} 

Where dilation rate is how may pixel are each filter spaced from each other, when dilation rate >= 1 then stride = 1 (since if you stride will be losing information from the data).

In [92]:
#### Hand tayloring filter size and deepness
##
#

filterlist = [8,8,8,8,8,16,16,16,16,16,32,32,32,32,32,47,47,32,32,32,32,32,16,16,16,16,16,4,4,4,4]

a = calculatefilter(600, filterlist)
print('First')
print(a)
print(len(filterlist))
print(10*'*')
filterlistb = [8,14,20,12,12,12,26,29,29,29,32,38,42,48,42,36,34,34,34,30,23,19,19]
b = calculatefilter(600, filterlistb)
print('Second')
print(b)
print(len(filterlistb))
print(10*'*')
filterlistc = [13,13,13,7,7,7,7,7,7,7,13,13,13,7,7,7,7,7,7,7,13,13,13,7,7,7,7,7,7,7,13,13,13,5,5,5,5,5,5,5,13,13,13,5,5,5,5,5,5,5,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,5,5,5,5,5,5,5,13,13,6]
c = calculatefilter(600, filterlistc)
print('Third')
print(c)
print(len(filterlistc))
print(10*'*')

filtering = [filterlist,filterlistb,filterlistc]

First
1
31
**********
Second
1
23
**********
Third
1
75
**********


In [158]:
#Cascading deepness networks mem size

def compute_deepsize(filterlist, channellist):
  dim = np.size(filterlist[0])  
  c = 0
  if dim == 1:    
    for filter in range(len(filterlist)):        
      c = c + 4*channellist[filter]*filterlist[filter]*channellist[filter+1]
  return (c*8.0/(1024.0*1024.0*1024.0))
    
def channeling_rule(channelin, filterlist):        
  channellist = [channelin]
  for filter in filterlist:
    channellist.append(channellist[-1] + filter + 3)  

  return channellist 

In [196]:
#build graph
# gcnn2d,softmax,losscrossentropy
#config1 = [ {"0_frames":4,"0_0":(10,30,40,2)}, {"1_1":(10,2,2,8), "1_2":(10,2,1,16), "1_3":(10,4,1,32), "1_4":(10,8,1,64), "1_5":(10,8,1,64)},{"1_labels":2},{"2_null":a},{"3_labels":2,"3_learningrate":0.005}]
buildgraph = Builder(dtype=tf.float32,batch=20,datasize=(600,5), num_input = 10, channels=1)

buildgraph.get_directives('gcnn2d')
buildgraph.get_directives('softmax')
#buildgraph.set_archname('frame')
buildgraph.get_directives('reducemean')
buildgraph.get_directives('losscrossentropy')

for i in range(10):

  filtery = 0
  filter0 = 8
  buildgraph.config_block_name(deepness=31,numblock=i)
  buildgraph.build_graph_module('gcnn2d', channels_out = 8,filter_size=(filter0,5), isinput=True, lastnamescope=True, show_cgraph=False, verbose=False)
 
  for j in np.arange(1,31):
    filtery= filterlist[j]  
    buildgraph.build_graph_module('gcnn2d', channels_out = channellist[j],filter_size=(filtery,1), isinput=False, lastnamescope=True, show_cgraph=False, verbose=False)    
    
  buildgraph.config_block_name(deepness=0,numblock=i)
  buildgraph.build_graph_module('softmax', num_labels=2, isinput=False, lastnamescope=True, show_cgraph=False)

'''
Avoiding the block reuse for awhile since tensorflow made it hard to keep
buildgraph.define_block('blocktest')
for i in np.arange(1,3):
  buildgraph.from_block('blocktest',tag='_'+str(i))
'''
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('reducemean', isinput=False, lastnamescope=True, show_cgraph=False)
buildgraph.config_block_name(deepness=0,numblock=0)
buildgraph.build_graph_module('losscrossentropy', num_labels = 2, batch=20, learningrate=0.005, isinput=False, lastnamescope=True, show_cgraph=False)

MemoryError: The model will allocate more memory, 27.4736517817, than the given limit.

In [124]:
print('scope count ',vscope.variable_scopes_count)

('scope count ', {u'losscrossentropy/gcnn2d_9_d26/conv/kernel/Adam': 0, u'losscrossentropy/gcnn2d_5_d12/conv/kernel/Adam': 0, u'losscrossentropy/gcnn2d_d20/conv/kernel/Adam_1': 0, u'losscrossentropy/gcnn2d_9_d8/conv/bias/Adam_1': 0, u'losscrossentropy/softmax_6/logit/biases/Adam': 0, u'losscrossentropy/gcnn2d_7_d12/conv_linear/kernel/Adam_1': 0, u'losscrossentropy/gcnn2d_7_d10/conv_linear/bias/Adam': 0, u'losscrossentropy/gcnn2d_9_d20/conv_linear/bias/Adam': 0, u'losscrossentropy/gcnn2d_5/conv_linear/kernel/Adam': 0, u'losscrossentropy/gcnn2d_8_d3/conv_linear/bias/Adam_1': 0, u'losscrossentropy/gcnn2d_4_d3/conv_linear/bias/Adam_1': 0, u'losscrossentropy/gcnn2d_7_d24/conv_linear/kernel/Adam': 0, u'losscrossentropy/gcnn2d_1_d27/conv/bias/Adam': 0, u'losscrossentropy/gcnn2d_9_d3/conv/bias/Adam_1': 0, u'losscrossentropy/gcnn2d_3_d27/conv/bias/Adam': 0, u'losscrossentropy/gcnn2d_1_d6/conv_linear/bias/Adam_1': 0, u'losscrossentropy/softmax_5/logit/weights/Adam_1': 0, u'losscrossentropy/gcnn2

In [125]:
print('variable ' ,vstore._vars)

('variable ', {u'losscrossentropy/gcnn2d_5_d25/conv/kernel/Adam_1/': <tf.Variable 'losscrossentropy/gcnn2d_5_d25/conv/kernel/Adam_1:0' shape=(16, 1, 16, 16) dtype=float32>, u'losscrossentropy/gcnn2d_8_d8/conv_linear/kernel/Adam_1/': <tf.Variable 'losscrossentropy/gcnn2d_8_d8/conv_linear/kernel/Adam_1:0' shape=(16, 1, 16, 16) dtype=float32>, u'losscrossentropy/gcnn2d_7_d24/conv_linear/bias/Adam/': <tf.Variable 'losscrossentropy/gcnn2d_7_d24/conv_linear/bias/Adam:0' shape=(16,) dtype=float32>, u'losscrossentropy/gcnn2d_3_d28/conv/bias/Adam/': <tf.Variable 'losscrossentropy/gcnn2d_3_d28/conv/bias/Adam:0' shape=(4,) dtype=float32>, u'losscrossentropy/gcnn2d_9_d7/conv_linear/kernel/Adam/': <tf.Variable 'losscrossentropy/gcnn2d_9_d7/conv_linear/kernel/Adam:0' shape=(16, 1, 16, 16) dtype=float32>, u'losscrossentropy/gcnn2d_3_d15/conv/kernel/Adam/': <tf.Variable 'losscrossentropy/gcnn2d_3_d15/conv/kernel/Adam:0' shape=(47, 1, 47, 47) dtype=float32>, u'losscrossentropy/gcnn2d_4_d11/conv/bias/Ad

In [186]:
buildgraph.graph._collections['variables'][-30:-1]

[<tf.Variable 'losscrossentropy/gcnn2d_9_d26/conv/bias/Adam:0' shape=(696,) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d26/conv/bias/Adam_1:0' shape=(696,) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv_linear/kernel/Adam:0' shape=(4, 1, 696, 703) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv_linear/kernel/Adam_1:0' shape=(4, 1, 696, 703) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv_linear/bias/Adam:0' shape=(703,) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv_linear/bias/Adam_1:0' shape=(703,) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv/kernel/Adam:0' shape=(4, 1, 696, 703) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv/kernel/Adam_1:0' shape=(4, 1, 696, 703) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv/bias/Adam:0' shape=(703,) dtype=float32>,
 <tf.Variable 'losscrossentropy/gcnn2d_9_d27/conv/bias/Adam_1:0' shape=(703,) dtype=floa

In [ ]:
# Label Vector Feed

lvec = []
for i in range(len(label_list)):
  label = label_list[i]  
  if label == 'sv' or label == 'SV':
    lvec.append([0,1]) 
    
  if label == 'cv' or label == 'CV':
    lvec.append([1,0])    

In [ ]:
# Run Computation Graph

feed_dict = {}
data_list = []
for i in range(10):  
  feed_dict[buildgraph.signal_in[i]] = data_tensor[:,:,5*i:5*(i+1),:] 

print(buildgraph.l_vec)

feed_dict[buildgraph.l_vec] = lvec  

output = buildgraph.run_cgraph(feed_dict, op_to_run = 'blocktest/gcnn2d_1:0', number_of_runs = 3, mode = 'minimize', new_session=True, output_log = True, verbose=True)
#print('Tensor {}'.format(buildgraph.namescopes.gcnn2d_1[1].name))
print(np.shape(output))
print(output)